In [1]:
import os
# import mapclassify
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import geopandas as geo
import ipywidgets as widgets
from IPython.display import display
from mpl_toolkits.axes_grid1 import make_axes_locatable
from datetime import datetime

%matplotlib inline

#### Import Dataset

In [2]:
root = 'C:/Users/Aaron/PycharmProjects/ECE-225'
data_path = Path(root + '/data/output.csv')
df = pd.read_csv(data_path)
print('Rate Max: ' + str(df.loc[:,'Rate'].max()))
print('Rate Min.: ' + str(df.loc[:,'Rate'].min()))
print('Rate Mean: ' + str(df.loc[:,'Rate'].mean()))
print('Rate Std.: ' + str(df.loc[:,'Rate'].std()))   

# df.loc[:,'Rate'] = ((df.loc[:,'Rate']-df.loc[:,'Rate'].mean())/df.loc[:,'Rate'].std())

Rate Max: 58.4
Rate Min.: 0.0
Rate Mean: 6.175009711500675
Rate Std.: 3.112534815614922


In [3]:
df['Date'] = df.apply(lambda x: x.Month+' '+str(x.Year), axis=1)
df.Date = pd.to_datetime(df.Date)

In [4]:
df.drop(columns=['Year', 'Month'], inplace=True)

In [5]:
df.head()

,State,County,Rate,Date
0,Mississippi,Newton County,6.1,2015-02-01
1,Mississippi,Panola County,9.4,2015-02-01
2,Mississippi,Monroe County,7.9,2015-02-01
3,Mississippi,Hinds County,6.1,2015-02-01
4,Mississippi,Kemper County,10.6,2015-02-01


#### Import .shp file data for counties

In [6]:
shp_counties = Path(root + "/data/UScounties/uscounties.shp")

c_map = geo.read_file(shp_counties)
c_map = c_map[c_map.STATE_NAME != 'Alaska']
c_map = c_map[c_map.STATE_NAME != 'Hawaii']
c_map.rename(columns={'NAME':'County', 'STATE_NAME':'State'}, inplace=True)
c_map.drop(columns=['STATE_FIPS', 'CNTY_FIPS', 'FIPS'], inplace=True)

df.convert_dtypes()
c_map.convert_dtypes()


,County,State,geometry
0,Lake of the Woods,Minnesota,"POLYGON ((-95.34283 48.54668, -95.34105 48.715..."
1,Ferry,Washington,"POLYGON ((-118.85163 47.94956, -118.84846 48.4..."
2,Stevens,Washington,"POLYGON ((-117.43883 48.04412, -117.54219 48.0..."
3,Okanogan,Washington,"POLYGON ((-118.97209 47.93915, -118.97406 47.9..."
4,Pend Oreille,Washington,"POLYGON ((-117.43858 48.99992, -117.03205 48.9..."
...,...,...,...
3104,LaGrange,Indiana,"POLYGON ((-85.66327 41.53557, -85.65946 41.762..."
3105,Steuben,Indiana,"POLYGON ((-84.79137 41.53049, -85.19715 41.537..."
3132,Pittsylvania,Virginia,"POLYGON ((-79.71720 36.54803, -79.64135 36.846..."
3133,Danville,Virginia,"POLYGON ((-79.51977 36.59777, -79.50138 36.621..."


In [7]:
c_map.head()

,County,State,geometry
0,Lake of the Woods,Minnesota,"POLYGON ((-95.34283 48.54668, -95.34105 48.715..."
1,Ferry,Washington,"POLYGON ((-118.85163 47.94956, -118.84846 48.4..."
2,Stevens,Washington,"POLYGON ((-117.43883 48.04412, -117.54219 48.0..."
3,Okanogan,Washington,"POLYGON ((-118.97209 47.93915, -118.97406 47.9..."
4,Pend Oreille,Washington,"POLYGON ((-117.43858 48.99992, -117.03205 48.9..."


In [8]:
# Clean Data
df['County'] = df.County.str.replace(' County', '')
print(df.head())
print(np.sum(df[['State', 'County']].isin({'County':['Newton'], 'State':['Mississippi']})))

         State  County  Rate       Date
0  Mississippi  Newton   6.1 2015-02-01
1  Mississippi  Panola   9.4 2015-02-01
2  Mississippi  Monroe   7.9 2015-02-01
3  Mississippi   Hinds   6.1 2015-02-01
4  Mississippi  Kemper  10.6 2015-02-01
State     26568
County     1523
dtype: int64


In [9]:
print(np.sum(c_map[['State', 'County']].isin({'County':['Newton'], 'State':['Mississippi']})))

State     82
County     6
dtype: int64


In [10]:
print('Unique Counties in Data: '+ str(np.array(df['County'].unique()).shape))
print('Uniqeu counties in SHP: '+ str(np.array(c_map['County'].unique()).shape))

Unique Counties in Data: (1752,)
Uniqeu counties in SHP: (1806,)


### Plot with Slider

In [11]:
print(df.head())

         State  County  Rate       Date
0  Mississippi  Newton   6.1 2015-02-01
1  Mississippi  Panola   9.4 2015-02-01
2  Mississippi  Monroe   7.9 2015-02-01
3  Mississippi   Hinds   6.1 2015-02-01
4  Mississippi  Kemper  10.6 2015-02-01


In [18]:
months = ["January", "February", "March","April", "May", "June", "July", "August","September", "October","November" , "December" ]

def plotmap(y, m):
    fig, ax = plt.subplots(1, figsize=(20,20))
    div = make_axes_locatable(ax)
    cax = div.append_axes("right", size="5%", pad=0.1)
    c_map.boundary.plot(ax=ax, markersize=0.0001, zorder=1)
    new = geo.geodataframe.GeoDataFrame(c_map.merge(df[(df.Date.dt.year == y) & (df.Date.dt.month == m)], on=['County', 'State'], how='left'), geometry='geometry')
    new.plot(column='Rate', ax=ax, legend=True, cax=cax, cmap='inferno', zorder=2)



In [24]:
# slider = widgets.IntSlider(min=1, max=(np.max(df.Date.dt.year)-np.min(df.Date.dt.year))*12)

year_s = widgets.IntSlider(min=np.min(df.Date.dt.year), max=np.max(df.Date.dt.year), description="Year", continuous_update=False)
mon_s = widgets.IntSlider(min=1, max=12, description="Month")
play = widgets.Play(
    value=1990,
    min=1990,
    max=2016,
    step=1,
    interval=500,
    description="Press play",
    disabled=False
)
widgets.jslink((play, 'value'), (year_s, 'value'))
widgets.interact(plotmap, y=year_s, m=mon_s)
widgets.HBox([play, year_s])

interactive(children=(IntSlider(value=1990, continuous_update=False, description='Year', max=2016, min=1990), …